In [27]:
import numpy as np
import tensorcircuit as tc
import tensorflow as tf
import math

def comp(c, i, j, k):
    c.ccnot(i, j, k)
    c.X(i)
    c.X(j)
    c.ccnot(i, j, k)
    c.X(i)
    c.X(j)
    return c

def oracle(c, n):
    for i in range(5):
        c = comp(c, i, i+1, i+6)
    c = comp(c, 0, 3, 11)
    c = comp(c, 2, 5, 12)
    c.multicontrol(*range(6, 14), unitary = tc.gates.x(), ctrl=[1 for _ in range(6, 13)])
    for i in range(5):
        c = comp(c, i, i+1, i+6)
    c = comp(c, 0, 3, 11)
    c = comp(c, 2, 5, 12)
    return c

def reflection(c, n):
    for i in range(n):
        c.H(i)
        c.X(i)
    c.multicontrol(*range(n), unitary = tc.gates.z(), ctrl=[1 for _ in range(n - 1)])
    for i in range(n):
        c.X(i)
        c.H(i)
    return c

In [36]:
n = 6
m = 2
tot_qubit = 14
theta = math.asin(1.0/(4*math.sqrt(2)))
r = round((math.pi / (2 * theta) - 1) / 2)
c = tc.Circuit(tot_qubit)

for i in range(6):
    c.H(i)
for i in range(6, tot_qubit-1):
    c.X(i)
c.H(tot_qubit-1)
c.Z(tot_qubit-1)
g = c.state()
for j in range(r):
    c = oracle(c, n)
    c = reflection(c, n)
c.Z(tot_qubit-1)
c.H(tot_qubit-1)
c.sample()

(array([1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.],
       dtype=float32),
 0.4995896855041375)